# 🧠 Agent Control Plane — GPU Judge (Llama 3.2:1b)

This notebook runs a local LLM-as-a-Judge on a Colab T4 GPU.  
It exposes a **FastAPI** endpoint via **Ngrok** that our MERN backend calls to evaluate task strategic alignment.

**Run Cell 1 first (setup), then Cell 2 (server + tunnel).** Copy the Ngrok URL printed at the bottom.

In [ ]:
# ═══════════════════════════════════════════════════════════════
# CELL 1 — Install Dependencies & Pull Model
# ═══════════════════════════════════════════════════════════════

# Install zstd (required by Ollama installer)
!apt-get update -qq && apt-get install -y -qq zstd > /dev/null 2>&1
print('✅ zstd installed')

# Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh
print('✅ Ollama installed')

# Install Python packages (pin uvicorn <0.32 to avoid loop_factory bug)
!pip install -q pyngrok fastapi 'uvicorn<0.32' nest-asyncio
print('✅ Python packages installed')

# Start Ollama server in background
import subprocess, time, os
os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
subprocess.Popen(
    ['ollama', 'serve'],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)
time.sleep(5)  # Wait for server to initialize
print('✅ Ollama server started')

# Pull the lightweight Llama 3.2:1b model (optimized for T4 GPU)
!ollama pull llama3.2:1b

print('\n✅ Setup complete — Ollama running with llama3.2:1b loaded.')
print('   Now run Cell 2 to start the Judge server + Ngrok tunnel.')

In [ ]:
# ═══════════════════════════════════════════════════════════════
# CELL 2 — FastAPI Judge Server + Ngrok Tunnel
# ═══════════════════════════════════════════════════════════════
#
# ⚠️  PASTE YOUR NGROK AUTH TOKEN BELOW BEFORE RUNNING THIS CELL
#

NGROK_AUTH_TOKEN = "YOUR_NGROK_TOKEN"  # <-- Replace this!

# ──────────────────────────────────────────────────────────────

import json, re, subprocess, threading, time
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

app = FastAPI(title='TaskEase Governance Judge')

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_methods=['*'],
    allow_headers=['*'],
)

# ── Request / Response schemas ─────────────────────────────────
class EvaluateRequest(BaseModel):
    taskDetails: str
    projectOKRs: str

class EvaluateResponse(BaseModel):
    approved: bool
    reason: str

# ── LLM helper ─────────────────────────────────────────────────
def query_llama(task_details: str, project_okrs: str) -> dict:
    """Send a structured prompt to llama3.2:1b via Ollama CLI and parse JSON."""

    prompt = (
        'You are a Strategic Governance Judge for a project management platform. '
        'Your job is to evaluate whether a proposed task is strategically aligned '
        "with the project's OKRs.\n\n"
        f'PROJECT OKRs:\n{project_okrs}\n\n'
        f'PROPOSED TASK:\n{task_details}\n\n'
        'INSTRUCTIONS:\n'
        '1. Evaluate whether this task contributes to or supports the stated OKRs.\n'
        '2. If the task is clearly unrelated, addresses a different goal, or would '
        'cause strategic drift, set approved to false.\n'
        '3. If the task supports, enables, or directly contributes to any OKR, '
        'set approved to true.\n'
        '4. Provide a concise reason (1-2 sentences) explaining your decision.\n\n'
        'You MUST respond with ONLY valid JSON in this exact format, no other text:\n'
        '{"approved": true, "reason": "your reason here"}\n'
        'or\n'
        '{"approved": false, "reason": "your reason here"}'
    )

    try:
        result = subprocess.run(
            ['ollama', 'run', 'llama3.2:1b', prompt],
            capture_output=True,
            text=True,
            timeout=60
        )
        raw = result.stdout.strip()
        print(f'[Judge Raw Output] {raw}')

        # Extract JSON from response (handles markdown fences or extra text)
        json_match = re.search(r'\{[^{}]*"approved"[^{}]*\}', raw)
        if json_match:
            parsed = json.loads(json_match.group())
            return {
                'approved': bool(parsed.get('approved', True)),
                'reason': str(parsed.get('reason', 'No reason provided.'))
            }

        # Fallback: approve if we can't parse (fail-open)
        return {'approved': True, 'reason': f'Judge response could not be parsed. Raw: {raw[:200]}'}

    except subprocess.TimeoutExpired:
        return {'approved': True, 'reason': 'Judge timed out — defaulting to approved.'}
    except Exception as e:
        return {'approved': True, 'reason': f'Judge error: {str(e)} — defaulting to approved.'}

# ── Endpoints ──────────────────────────────────────────────────
@app.post('/evaluate-task', response_model=EvaluateResponse)
async def evaluate_task(req: EvaluateRequest):
    """Evaluate a proposed task against project OKRs using local LLM."""
    if not req.taskDetails or not req.projectOKRs:
        raise HTTPException(status_code=400, detail='taskDetails and projectOKRs are required.')

    result = query_llama(req.taskDetails, req.projectOKRs)
    print(f'[Judge Decision] approved={result["approved"]} | reason={result["reason"]}')
    return EvaluateResponse(**result)

@app.get('/health')
async def health():
    return {'status': 'ok', 'model': 'llama3.2:1b'}

# ── Start FastAPI in background thread ─────────────────────────
import uvicorn

def start_server():
    """Run uvicorn in a way compatible with nest_asyncio."""
    config = uvicorn.Config(app, host='0.0.0.0', port=8000, log_level='info')
    server = uvicorn.Server(config)
    import asyncio
    loop = asyncio.new_event_loop()
    loop.run_until_complete(server.serve())

server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()
time.sleep(3)  # Let server start
print('🚀 FastAPI Judge Server running on port 8000')

# ── Ngrok tunnel ──────────────────────────────────────────────
from pyngrok import ngrok

ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(8000)

print('\n' + '=' * 60)
print('🌐 NGROK TUNNEL ACTIVE')
print(f'   Public URL: {public_url}')
print(f'   Test:  GET  {public_url}/health')
print(f'   Judge: POST {public_url}/evaluate-task')
print('=' * 60)
print('\n📋 Copy the URL above and paste it back to the AI assistant.')
print('   It will be set as COLAB_JUDGE_URL in your .env file.\n')

# Keep cell alive so the server and tunnel stay running
try:
    while True:
        time.sleep(1)  
except KeyboardInterrupt:
    print('\nShutting down...')
    ngrok.kill()

: 